<a href="https://colab.research.google.com/github/frosDfurret/shopscoop/blob/main/shopscoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <- Click to start
#@markdown Running this will take a while, so please be patient. When scooping is finished, the output will automatically download as "output.zip".<br>This tool has only been tested with the 3DS eShop. Use with other eShops may require modifications to the code.<br><tt>regions: {AD, AE, AG, AI, AL, AN, AR, AT, AU, AW, AZ, BA, BB, BE, BG, BM, BO, BR, BS, BW, BZ, CA, CH, CL, CO, CR, CY, CZ, DE, DJ, DK, DM, DO, EC, EE, ER, ES, FI, FR, GB, GD, GF, GG, GI, GP, GR, GT, GY, HK, HN, HR, HT, HU, IE, IM, IN, IS, IT, JE, JM, JP, KN, KR, KY, LC, LI, LS, LT, LU, LV, MC, ME, MK, ML, MQ, MR, MS, MT, MX, MY, MZ, NA, NE, NI, NL, NO, NZ, PA, PE, PL, PT, PY, RO, RS, RU, SA, SD, SE, SG, SI, SK, SM, SO, SR, SV, SZ, TC, TD, TR, TT, TW, US, UY, VA, VC, VE, VG, VI, ZA, ZM, ZW}<br>lang: {JP,EN,FR,DE,IT,ES,ZH,KO}<br>shop_id: {1,2,3}</tt>
import xml.etree.ElementTree as xml
import os
import requests
import progressbar
import unicodedata
import re
import time
from google.colab import files
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

print("shopscoop")
print("by daguil")
print("disclaimer: this python script is designed to run in colab.\nrunning this script elsewhere *will* require modifications to the code.")
print("---------------------------------------")
def get(url):
  return(requests.get(url,verify=False))
def save(content, path, t):
  file = open(path, t)
  file.write(content)
  file.close()
def slugify(value):
    #Taken from https://github.com/django/django/blob/master/django/utils/text.py
    value = str(value)
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')
id = []
offset = 0
complete = False
region = 'JP' #@param {type:"string"}
language = 'JP' #@param {type:"string"}
shop_id = '1' #@param {type:"string"}
if not os.path.exists('/content/files'):
  print('- Creating directories')
  os.mkdir('/content/files')
  os.mkdir('/content/files/images')
  os.mkdir('/content/files/index')
  os.mkdir('/content/files/raw')

print('- Scooping indexes')
while complete == False:
  g = get('https://samurai.ctr.shop.nintendo.net/samurai/ws/'+region+'/titles?shop_id='+shop_id+'&limit=1000&lang='+language+'&offset='+str(offset))
  root = xml.fromstring(g.text)
  contents = root.find('contents')

  for i in contents.findall('content'):
    currentid = i.find('title').get('id')
    id.append(currentid)
  
  save(g.text, '/content/files/index/'+str(int(offset/1000))+'.txt','w')
  print('  - Scooped index '+str(int(offset/1000)))
  if contents.get('length') == '1000':
    offset += 1000
  else:
    complete = True

print('- Finished scooping all indexes')
print('- Found '+contents.get('total')+' titles')
print('- Scooping store pages')
time.sleep(1)
for i in progressbar.progressbar(range(len(id))):
  g = get('https://samurai.ctr.shop.nintendo.net/samurai/ws/'+region+'/title/'+id[i]+'?shop_id='+shop_id+'&lang='+language)
  root = xml.fromstring(g.text)
  title = root.find('title').find('name').text
  title = title.replace('\n','').replace('<br>','')
  idtitle = id[i]+' '+title
  idonly = id[i]
  iconurl = get(root.find('title').find('icon_url').text)
  bannerurl = get(root.find('title').find('banner_url').text)
  tburls = []
  if not root.find('title').find('thumbnails') == None:
      for i in root.find('title').find('thumbnails').findall('thumbnail'):
        tburls.append(get(i.get('url')))
      for i in range(len(tburls)):
        save(tburls[i].content, '/content/files/images/'+slugify(idtitle+' tb'+str(i))+'.jpg','wb')
  sstopurls = []
  ssbottomurls = []
  if not root.find('title').find('screenshots') == None:
    for i in root.find('title').find('screenshots').findall('screenshot'):
      sstopurls.append(get(i[0].text))
      ssbottomurls.append(get(i[1].text))
    for i in range(len(sstopurls)):
      save(sstopurls[i].content, '/content/files/images/'+slugify(idtitle+' sstop'+str(i))+'.jpg','wb')
    for i in range(len(ssbottomurls)):
      save(ssbottomurls[i].content, '/content/files/images/'+slugify(idtitle+' ssbottom'+str(i))+'.jpg','wb')
  save(g.text, '/content/files/raw/'+slugify(idtitle)+'.txt','w')
  save(iconurl.content, '/content/files/images/'+slugify(idtitle+' icon')+'.jpg','wb')
  save(bannerurl.content, '/content/files/images/'+slugify(idtitle+' banner')+'.jpg','wb')
  save('\n'+slugify(idtitle), '/content/files/filenameindex.txt', 'a')
  save('\n'+idtitle, '/content/files/realnameindex.txt', 'a')
  save('\n'+idonly, '/content/files/idonlyindex.txt', 'a')
  save('\n'+title, '/content/files/titleonlyindex.txt', 'a')
  #os.system('cls')
print('- Finished scooping store pages')
print('- Zipping files')
os.system('zip -r /content/output.zip /content/files/')
print('- Initiating download seqence')
files.download ('/content/output.zip')
print('- ALL DONE!!')